In [28]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
 
    
try:
    df = pd.read_csv('../debug_programados_del_dia.csv', encoding='utf-8',  sep='|', engine='python')
    print(df.head())
except Exception as e:
    print(f"An error occurred: {e}")

   doc_id    doc_emp  doc_num_gr doc_num_fac             doc_fec_crea  \
0   84505  FIAMASTER   T002-1940         NaN  2025-08-20 08:26:47.714   
1   84468  FIAMASTER   T002-1941         NaN  2025-08-19 14:47:37.105   
2   84479  FIAMASTER   T002-1942         NaN  2025-08-19 15:25:25.441   
3   84487  FIAMASTER   T002-1943         NaN  2025-08-19 16:11:48.480   
4   84449  FIAMASTER  T007-36511  F007-68839  2025-08-19 11:36:01.755   

  doc_fec_ent_gr doc_fec_prog_gr           doc_raz_soc  \
0     2025-08-20      2025-08-20        FIAMASTER S.A.   
1     2025-08-20      2025-08-20        FIAMASTER S.A.   
2     2025-08-20      2025-08-20        FIAMASTER S.A.   
3     2025-08-20      2025-08-20        FIAMASTER S.A.   
4     2025-08-20      2025-08-15  COMERCIAL ANGULO SRL   

                        doc_con_pag  doc_ag_trans  ...          rd_hora_sal  \
0  NO USAR - Contado contra entrega           NaN  ...  2025-08-20 12:29:56   
1  NO USAR - Contado contra entrega           NaN  ...

In [ ]:
def save_dataframe_as_image(df, title, filename="reporte_despacho.png"):
    """Renderiza el DataFrame como una tabla en Matplotlib y la guarda como imagen."""
    
    # Crea una figura y un eje
    # Ajusta el tamaño de la figura en función del número de filas para evitar el recorte
    fig, ax = plt.subplots(figsize=(7, len(df) * 0.5)) 
    ax.axis('tight')
    ax.axis('off')

    # Convierte todos los datos a string para la representación de la tabla
    cell_data = df.values.astype(str)
    
    # Crea la tabla de matplotlib
    table = ax.table(cellText=cell_data,
                     colLabels=df.columns,
                     loc='center',
                     cellLoc='center')

    # Estilizado
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    
    border_color = "#C7D2DD" # Gris azulado suave
    header_bg_color = '#4682B4' # Azul fuerte para la cabecera
    total_row_bg_color = '#E0E0E0' # Gris un poco más oscuro para la fila total
    
    
    # Aplicar estilos a la cabecera
    for (row, col), cell in table.get_celld().items():
        cell.set_edgecolor(border_color) # Borde para todas las celdas
        
        if row == 0: # Cabecera
            cell.set_text_props(weight='bold', color='white')
            cell.set_facecolor(header_bg_color) 
            # Borde más pronunciado para la cabecera si se desea
            cell.set_linewidth(1.5) 
        elif row == len(df): # Fila 'Total general'
            cell.set_text_props(weight='bold')
            cell.set_facecolor(total_row_bg_color) 
            cell.set_linewidth(1.5) # Borde más pronunciado para la fila total
        else: # Celdas de datos
            cell.set_facecolor('white') # Fondo blanco para las celdas de datos
            cell.set_linewidth(0.5) # Bordes más finos para las celdas de datos
            
    # Ajuste para los bordes externos de toda la tabla si se necesitan más gruesos
    # Esto puede hacerse configurando los bordes del propio objeto table, o
    # asegurando que las celdas en los bordes externos tengan un linewidth mayor.
    # Por ejemplo, para la caja exterior:
    for i in range(len(df)):
        for j in range(len(df.columns)):
            cell = table[(i, j)]
            if i == 0: # Borde superior de la tabla
                cell.set_linewidth(1.5)
            if i == len(df) : # Borde inferior de la tabla
                cell.set_linewidth(1.5)
            if j == 0: # Borde izquierdo de la tabla
                cell.set_linewidth(1.5)
            if j == len(df.columns) : # Borde derecho de la tabla
                cell.set_linewidth(1.5)


    # Agregar título
    plt.title(title, fontsize=14, fontweight='bold', pad=15)

    # Guardar la imagen (El entorno de ejecución la mostrará)
    plt.savefig(filename, bbox_inches='tight', dpi=150)
    plt.close(fig) 
    # Imprimir el DataFrame para verificación en la consola
    print("--- Contenido del DataFrame antes de guardar como imagen (Verificación) ---")
    print(df.to_string(index=False))
    print("\nReporte guardado como 'reporte_despacho.png' y mostrado a continuación.")
    

In [46]:

# Limpieza de datos (Asegurando que los NaN sean 0 para la suma)
import datetime


df['doc_peso_tot'] = pd.to_numeric(df['doc_peso_tot'], errors='coerce').fillna(0)
df['rut_id'] = pd.to_numeric(df['rut_id'], errors='coerce').fillna(0)

df_grouped = df.groupby('veh_placa').agg(
    Peso_Total=('doc_peso_tot', 'sum'),
    cantidad_viajes=('rut_id', 'nunique') 
).reset_index()

# 2.2. Renombrar la columna de vehículos (para coincidir con la imagen)
df_grouped = df_grouped.rename(columns={'veh_placa': 'Vehiculos'})

# 2.3. Calcular el Peso Total General
total_general_peso = df_grouped['Peso_Total'].sum()

# 2.4. Calcular el Porcentaje
df_grouped['Avance %'] = (df_grouped['Peso_Total'] / total_general_peso)

 
# 3.1. Ordenar por el nombre del vehículo
df_grouped = df_grouped.sort_values(by='Peso_Total').reset_index(drop=True)

# 3.2. Crear el Total General (fila)
# ************** CORRECCIÓN CLAVE: Usar df_grouped, no df **************
# Calcular el Total General de Viajes sumando la columna ya agrupada.
total_rutas = df_grouped['cantidad_viajes'].sum() 

total_row = pd.DataFrame([{
    'Vehiculos': 'Total general',
    'Peso_Total': total_general_peso,
    'Avance %': 1.0,
    # Usamos el mismo nombre de columna que en df_grouped para estandarizar
    'cantidad_viajes': total_rutas 
}])

# 3.3. Concatenar la fila de total
df_final = pd.concat([df_grouped, total_row], ignore_index=True)


def format_row(row):
    # Formatear el peso
    if row['Vehiculos'] == 'Total general':
        # Mostrar el total en toneladas (X.Xt)
        peso_str = f"{row['Peso_Total'] / 1000:.1f} t"
 
    elif row['Peso_Total'] >= 1000:
        # Mostrar pesos de 1000 a 1999 en toneladas
        peso_str = f"{row['Peso_Total'] / 1000:.2f} t"
    else:
        # Mostrar pesos pequeños en kg sin decimales
        peso_str = f"{row['Peso_Total']:.0f} kg"
        
    # Formatear el porcentaje
    porcentaje_str = f"{row['Avance %']:.2%}"
    
    # Formatear la Cantidad de viajes (ahora la columna es 'cantidad_viajes' en todos los casos)
    ruta_val = int(row['cantidad_viajes'])

    return pd.Series({
        'Vehiculos': row['Vehiculos'],
        'Peso Total': peso_str,
        'Avance %': porcentaje_str, # Nombre de columna de la imagen
        'Cant. de viajes': ruta_val # Usamos 'Cantidad de viajes' para coincidir con la imagen final
    })

# Aplicar el formato
# Renombramos las columnas del DF estilizado para usar los nombres de la imagen (%, Ruta)
df_styled = df_final.apply(format_row, axis=1)

# --- 4. Resultado (Impresión) ---
report_title = f"DESPACHOS"

print(df_styled.to_string(index=False))

save_dataframe_as_image(df_styled, report_title)

    Vehiculos Peso Total Avance %  Cant. de viajes
      BSQ-823     631 kg    5.49%                1
      CCA-828     1.65 t   14.33%                1
      BSR-797     1.99 t   17.32%                1
      BSS-818     2.20 t   19.11%                1
      CCB-949     2.20 t   19.11%                1
      BSS-795     2.83 t   24.64%                2
Total general     11.5 t  100.00%                7
--- Contenido del DataFrame antes de guardar como imagen (Verificación) ---
    Vehiculos Peso Total Avance %  Cant. de viajes
      BSQ-823     631 kg    5.49%                1
      CCA-828     1.65 t   14.33%                1
      BSR-797     1.99 t   17.32%                1
      BSS-818     2.20 t   19.11%                1
      CCB-949     2.20 t   19.11%                1
      BSS-795     2.83 t   24.64%                2
Total general     11.5 t  100.00%                7

Reporte guardado como 'reporte_despacho.png' y mostrado a continuación.


In [149]:
import random
import colorsys

def hsl_to_hex(h, s, l):
    """Convierte valores HSL (0-1) a HEX."""
    r, g, b = colorsys.hls_to_rgb(h, l, s)
    return "#{:02X}{:02X}{:02X}".format(int(r * 255), int(g * 255), int(b * 255))

def generate_dynamic_neon_palette(n_colors=15, mode="light"):
    """
    Genera colores estilo NEÓN dinámicos, adaptados para modo LIGHT o DARK.
    Inspirado en el uso de Accent Colors de SAP Fiori Android.
    """
    palette = []

    for _ in range(n_colors):
        # Hue en todo el espectro (0–1)
        h = random.random()

        if mode == "light":
            s = random.uniform(0.70, 0.95)  # saturación vibrante
            l = random.uniform(0.50, 0.70)  # luminosidad media-alta
        elif mode == "dark":
            s = random.uniform(0.80, 1.00)  # aún más saturado
            l = random.uniform(0.35, 0.55)  # luminosidad media-baja
        else:
            raise ValueError("El modo debe ser 'light' o 'dark'")

        palette.append(hsl_to_hex(h, s, l))

    random.shuffle(palette)

    return palette

# EJEMPLOS:
colors_light = generate_dynamic_neon_palette(15, mode="light")
colors_dark = generate_dynamic_neon_palette(15, mode="dark")

print("Light palette:", colors_light)
print("Dark palette:", colors_dark)


Light palette: ['#ED5058', '#EFB53F', '#E15D20', '#71F326', '#F41DEB', '#9BE23B', '#60E872', '#CAE423', '#6C63EE', '#7821F2', '#EC75CF', '#E76B6D', '#F4D41C', '#4DF1A5', '#44C3F0']
Dark palette: ['#17D2F1', '#AC11E0', '#882CEC', '#24F252', '#E2A310', '#C61C02', '#CAC314', '#B6200E', '#CBA900', '#F51363', '#BD570E', '#AE880D', '#A80F29', '#0690C9', '#F2831A']


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import os
 

 


class Donut:
    def __init__(self, value: float, label: str):
        self.value = value
        self.label = label


def generar_donut(
    valores: list[int],
    colores: list[str],
    filename: str,
    size: tuple[int, int] = (4, 4),
) -> None:
    fig, ax = plt.subplots(figsize=size, dpi=100)
    ax.pie(valores, colors=colores, wedgeprops={"width": 0.30}, startangle=90)
    ax.set(aspect="equal")
    plt.savefig(filename, transparent=True, bbox_inches="tight")
    plt.close()


def generar_donut_and_barra(
    img: Image.Image,
    draw: ImageDraw.ImageDraw,
    values: list[Donut],
    position: tuple[int, int] = (10, 80),
):
    name_image_ramdon = f"donut_{random.randint(1, 1000)}.png"
    colors_light = generate_dynamic_neon_palette(len(values), mode="light")
    _list_values = [f.value for f in values]
    generar_donut(_list_values, colors_light, name_image_ramdon, (3, 3))
    donut = Image.open(name_image_ramdon)

    draw.text(tuple(x + y for x, y in zip(position, (90, 85))), str(int(sum(_list_values))), fill="#3A3A3A", font=font_big)
    draw.text(tuple(x + y for x, y in zip(position, (100, 140))), "Total", fill="#777777", font=font_subtitle)

    x = 260 + position[0]
    y = 80 + position[1]



  

    img.paste(donut, position, donut)

    os.remove(name_image_ramdon)

  
    indicadores = [
        (
            str(values[i].value),
            values[i].label,
            f"{(values[i].value / sum(_list_values) * 100):.2f}%",
            colors_light[i],
        )
        for i in range(len(values))
    ]



    font_val = ImageFont.truetype("ARIAL.ttf", 28)
    font_label = font_small
    font_pct = ImageFont.truetype("ARIAL.ttf", 22)

    for val, label, pct, color in indicadores:
        # ------------------------------------------------
        # 1. Calcular ancho dinámico del bloque horizontal
        # ------------------------------------------------
        bbox_val = draw.textbbox((0, 0), val, font=font_val)
        bbox_label = draw.textbbox((0, 0), label, font=font_label)
        bbox_pct = draw.textbbox((0, 0), pct, font=font_pct)

        text_width = max(
            bbox_val[2] - bbox_val[0],
            bbox_label[2] - bbox_label[0],
            bbox_pct[2] - bbox_pct[0],
        )

        # Margen interno + espacio para barra vertical
        bloque_width = text_width + 30  # 15 (barra→texto) + 35 margen

        # ------------------------------------------------
        # 2. Dibujar barra vertical izquierd
        # ------------------------------------------------
        draw.line((x, y, x, y + 55), fill=color, width=6)

        # ------------------------------------------------
        # 3. Dibujar textos
        # ------------------------------------------------
        draw.text((x + 15, y), val, fill=color, font=font_val)
        draw.text((x + 15, y + 32), label, fill="#666666", font=font_label)
        draw.text((x + 15, y + 60), pct, fill="#777777", font=font_pct)

        # ------------------------------------------------
        # 4. Mover el cursor horizontalmente según ancho
        # ------------------------------------------------
        x += bloque_width



def generar_donut_and_barra_vertical(
    img: Image.Image,
    draw: ImageDraw.ImageDraw,
    values: list[Donut],
    position: tuple[int, int] = (10, 80),
):
    name_image_ramdon = f"donut_{random.randint(1, 1000)}.png"
    colors_light = generate_dynamic_neon_palette(len(values), mode="light")
    _list_values = [f.value for f in values]
    generar_donut(_list_values, colors_light, name_image_ramdon, (3, 3))
    donut = Image.open(name_image_ramdon)

    draw.text(tuple(x + y for x, y in zip(position, (90, 85))), str(int(sum(_list_values))), fill="#3A3A3A", font=font_big)
    draw.text(tuple(x + y for x, y in zip(position, (100, 140))), "Total", fill="#777777", font=font_subtitle)

    x = 40 + position[0]
    y = 250 + position[1]

    img.paste(donut, position, donut)

    os.remove(name_image_ramdon)

  
    indicadores = [
        (
            str(values[i].value),
            values[i].label,
            f"{(values[i].value / sum(_list_values) * 100):.2f}%",
            colors_light[i],
        )
        for i in range(len(values))
    ]



    font_val = ImageFont.truetype("ARIAL.ttf", 28)
    font_label = font_small
    font_pct = ImageFont.truetype("ARIAL.ttf", 22)

    for val, label, pct, color in indicadores:
        # ------------------------------------------------
        # 1. Calcular ancho dinámico del bloque horizontal
        # ------------------------------------------------
        bbox_val = draw.textbbox((0, 0), val, font=font_val)
        bbox_label = draw.textbbox((0, 0), label, font=font_label)
        bbox_pct = draw.textbbox((0, 0), pct, font=font_pct)

        text_width = max(
            bbox_val[2] - bbox_val[0],
            bbox_label[2] - bbox_label[0],
            bbox_pct[2] - bbox_pct[0],
        )

        # Margen interno + espacio para barra vertical
        bloque_width = text_width + 30  # 15 (barra→texto) + 35 margen

        # ------------------------------------------------
        # 2. Dibujar barra vertical izquierd
        # ------------------------------------------------
        draw.line((x, y, x, y + 55), fill=color, width=6)

        # ------------------------------------------------
        # 3. Dibujar textos
        # ------------------------------------------------
        draw.text((x + 15, y), val, fill=color, font=font_val)
        draw.text((x + 15, y + 32), label, fill="#666666", font=font_label)
        draw.line((x + bloque_width, y+30, x + bloque_width, y + 50), fill="#E5E7EB", width=2)
        draw.text((x + bloque_width + 15, y + 32), pct, fill="#777777", font=font_pct)

        # ------------------------------------------------
        # 4. Mover el cursor horizontalmente según ancho
        # ------------------------------------------------
        y += 75


  


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np


def peso_legible(valor):
    """
    Convierte cualquier entrada (texto o número) de kg a formato legible en toneladas o kg.
    Ejemplos:
        peso_legible("15420")     → "15 ton"
        peso_legible("50,000")    → "50 ton"
        peso_legible(8750)        → "8.75 ton"
        peso_legible("980")       → "0.98 ton"
        peso_legible(85)          → "85 kg"
    """
    # 1. Convertir a string y limpiar
    texto = str(valor).strip()

    # 2. Remover comas y espacios (soporta "15,420" o "15 420")
    texto_limpio = texto.replace(",", "").replace(" ", "")

    # 3. Si hay punto decimal (ej: "15.5" o "1,234.56")
    if "." in texto_limpio:
        # Separar parte entera y decimal
        partes = texto_limpio.split(".")
        entero = partes[0]
        decimal = partes[1] if len(partes) > 1 else "0"
        texto_limpio = entero + "." + decimal

    # 4. Convertir a float (ahora es seguro)
    try:
        kg = float(texto_limpio)
    except ValueError:
        return "Error: valor no válido"

    # 5. Convertir a toneladas
    toneladas = kg / 1000.0

    # 6. Formato inteligente
    if toneladas >= 10:
        return f"{toneladas:,.0f} ton".replace(
            ",", " "
        )  # opcional: espacio como separador
    elif toneladas >= 1:
        return f"{toneladas:,.2f} ton".replace(",", " ")
    else:
        return f"{kg:,.0f} kg".replace(",", " ")


WIDTH = 1920
HEIGHT = 1080

# === 1. Crear fondo blanco ===
img = Image.new("RGB", (WIDTH, HEIGHT), "white")
draw = ImageDraw.Draw(img)

# === 2. Cargar fuentes profesionales ===
font_title = ImageFont.truetype("ARIAL.ttf", 30)
font_subtitle = ImageFont.truetype("ARIAL.ttf", 24)
font_small = ImageFont.truetype("ARIAL.ttf", 18)
font_big = ImageFont.truetype("ARIAL.ttf", 48)

# === 3. Dibujar títulos ===
draw.text((50, 40), "Estado de vehículos", fill="#3A3A3A", font=font_title)



generar_donut_and_barra(
    img,
    draw,
    [
        Donut(value=1, label="Libres"),
        Donut(value=5, label="En comisión"),
        Donut(value=2, label="En ruta"),
    ],
)


vehiculos = [
    {
        "placa": "BSR-797",
        "modelo": "PEUGEOT BOXER",
        "porcentaje_del_tal": "12%",
        "total_vueltas": 2,
        "capacidad_x_placa": 1200,
        "capacidad_despachado": 960,
        "capacidad_despachado_porcentaje": "80%",
        "estatus": "Libre",
    },
    {
        "placa": "BSS-795",
        "modelo": "PEUGEOT BOXER",
        "porcentaje_del_tal": "18%",
        "total_vueltas": 3,
        "capacidad_x_placa": 1000,
        "capacidad_despachado": 1150,
        "capacidad_despachado_porcentaje": "115%",
        "estatus": "Libre",
    },
    {
        "placa": "CCA-932",
        "modelo": "PEUGEOT BOXER",
        "porcentaje_del_tal": "9%",
        "total_vueltas": 1,
        "capacidad_x_placa": 1500,
        "capacidad_despachado": 750,
        "capacidad_despachado_porcentaje": "50%",
        "estatus": "En ruta",
    },
    {
        "placa": "BSQ-823",
        "modelo": "PEUGEOT BOXER",
        "porcentaje_del_tal": "21%",
        "total_vueltas": 4,
        "capacidad_x_placa": 1300,
        "capacidad_despachado": 1690,
        "capacidad_despachado_porcentaje": "130%",
        "estatus": "En comisión",
    },
    {
        "placa": "BJU-859",
        "modelo": "PEUGEOT BOXER",
        "porcentaje_del_tal": "6%",
        "total_vueltas": 1,
        "capacidad_x_placa": 1400,
        "capacidad_despachado": 980,
        "capacidad_despachado_porcentaje": "70%",
        "estatus": "En mantenimiento",
    },
]
# Configuración visual
ROW_HEIGHT = 90
START_Y = 380
START_X = 20
ICON_X = START_X
TEXT_START_X = START_X + 5
WIDTH = 820 + START_X

# Fuentes
font_title = ImageFont.truetype("Arial.ttf", 24)
font_normal = ImageFont.truetype("Arial.ttf", 20)
font_badge = ImageFont.truetype("Arial.ttf", 16)

# Colores como en la imagen
COLORS = {
    "won": "#10B981",  # Verde
    "negotiation": "#3B82F6",  # Azul
    "libre": "#10B981",
    "en_ruta": "#F59E0B",
    "en_comision": "#8B5CF6",
    "en_mantenimiento": "#EF4444",
    "bg_alt": "#F9FAFB",  # Fondo fila alternada
    "separator": "#E5E7EB",
    "text_primary": "#111827",
    "text_secondary": "#6B7280",
}

# Mapeo de estatus a colores
status_colors = {
    "Libre": COLORS["libre"],
    "En ruta": COLORS["en_ruta"],
    "En comisión": COLORS["en_comision"],
    "En mantenimiento": COLORS["en_mantenimiento"],
}


y = START_Y


# header
draw.text(
    (START_X + 500, (y - ROW_HEIGHT) + 55),
    "Rutas",
    fill=COLORS["text_secondary"],
    font=font_title,
)
draw.text(
    (START_X + 730, (y - ROW_HEIGHT) + 55),
    "Fecha",
    fill=COLORS["text_secondary"],
    font=font_title,
)
draw.line(
    [(START_X, (y - ROW_HEIGHT) + 86), (WIDTH, (y - ROW_HEIGHT) + 86)],
    fill=COLORS["separator"],
    width=5,
)

for i, veh in enumerate(vehiculos):
    row_y = y + i * ROW_HEIGHT

    if i % 2 != 1:
        draw.rectangle([0, row_y, WIDTH, row_y + ROW_HEIGHT], fill=COLORS["bg_alt"])

    # Línea divisoria inferior sutil
    draw.line(
        [(START_X, row_y + ROW_HEIGHT), (WIDTH, row_y + ROW_HEIGHT)],
        fill=COLORS["separator"],
        width=1,
    )

    # === Placa (nombre principal) ===
    draw.text(
        (TEXT_START_X, row_y + 18),
        veh["placa"],
        fill=COLORS["text_primary"],
        font=font_title,
    )

    # === Modelo (subtexto gris) ===
    draw.text(
        (TEXT_START_X, row_y + 54),
        veh["modelo"],
        fill=COLORS["text_secondary"],
        font=font_small,
    )

    # === Badge de Estatus ===
    status = veh["estatus"]
    badge_color = status_colors.get(status, "#6B7280")
    badge_text = status.upper()

    # Fondo del badge
    badge_padding = 12
    text_bbox = draw.textbbox((0, 0), badge_text, font=font_badge)
    badge_width = text_bbox[2] - text_bbox[0] + badge_padding * 2
    badge_height = 25

    # badge de capacidad x placa
    badge_color_capacidad_x = COLORS["text_secondary"]
    badge_capacidad_x = START_X + 130
    badge_text_capacidad_x = f"{peso_legible(veh['capacidad_despachado'])} / {peso_legible(veh['capacidad_x_placa'])}"
    text_bbox_capacidad_x = draw.textbbox(
        (0, 0), badge_text_capacidad_x, font=font_badge
    )
    badge_width_capacidad_x = (
        text_bbox_capacidad_x[2] - text_bbox_capacidad_x[0] + badge_padding * 2
    )
    draw.rounded_rectangle(
        [
            badge_capacidad_x,
            row_y + 15,
            badge_capacidad_x + badge_width_capacidad_x,
            row_y + 15 + badge_height,
        ],
        radius=20,
        # fill=badge_color + "20",
        outline=badge_color_capacidad_x,
        width=2,
    )
    draw.text(
        (badge_capacidad_x + badge_padding, row_y + 28),
        badge_text_capacidad_x,
        fill=badge_color_capacidad_x,
        font=font_badge,
        anchor="lm",
    )

    # estado de vehiculo
    badge_x = START_X + 320
    draw.rounded_rectangle(
        [badge_x, row_y + 20, badge_x + badge_width, row_y + 20 + badge_height],
        radius=20,
        # fill=badge_color + "20",
        outline=badge_color,
        width=2,
    )
    draw.text(
        (badge_x + badge_padding, row_y + 20 + 15),
        badge_text,
        fill=badge_color,
        font=font_badge,
        anchor="lm",
    )

    # === Capacidad despachada ===
    monto = f"{veh['total_vueltas']}"
    draw.text(
        (START_X + 550, row_y + 32),
        monto,
        fill=COLORS["text_primary"],
        font=font_normal,
    )

    # === Fecha ===
    fecha_inicio = "Salida, Hoy 11:30"
    draw.text(
        (START_X + 800, row_y + 32),
        fecha_inicio,
        fill=COLORS["text_secondary"],
        font=font_small,
        anchor="rm",
    )

    fecha_fin = "Regreso, Hoy 11:30"
    draw.text(
        (START_X + 800, row_y + 55),
        fecha_fin,
        fill=COLORS["text_secondary"],
        font=font_small,
        anchor="rm",
    )


# === 8. Total de capacidad ===
draw.text(
    (START_X, row_y + 110), "Total de capacidad 13.1 t", fill="#555555", font=font_small
)
draw.text((550, 10), "Actualizado 27/11/25 12:30", fill="#555555", font=font_small)

# === 9. Lado derecho: Programación del día ===

# generar_donut([736, 89], ["#22A7F0", "#F24E71"], "donut2.png")
# donut2 = Image.open("donut2.png")
# img.paste(donut2, (1100, 400), donut2)
donut = Image.open("carga_laboral_close_style.png")
img.paste(donut, (880, 40), donut)


# === Donut pequeño ===
draw.rounded_rectangle(
    [900, 500, 1600, 780],
    radius=20,
    fill="#F9FAFB",
    outline="#E5E7EB",
    width=2,
)

draw.text(
    (920, 520),
    "PROGRAMACIÓN DEL DÍA",
    fill=COLORS["text_secondary"],
    font=font_title,
)
draw.text((920, 545), "27/11/25", fill="#999999", font=font_subtitle)

generar_donut_and_barra(
    img,
    draw,
    [
        Donut(value=1, label="Libres"),
        Donut(value=5, label="En comisión"),
        Donut(value=2, label="En ruta"),
    ],
    (900, 550),
)


draw.rounded_rectangle(
    [900, 800, 1600, 1050],
    radius=20,
    fill="#F9FAFB",
    outline="#E5E7EB",
    width=2,
)
generar_donut_and_barra(
    img,
    draw,
    [
        Donut(value=21, label="24 Horas"),
        Donut(value=35, label="48 Horas"),
        Donut(value=26, label="72 Horas"),
        Donut(value=43, label="Otros"),
    ],
    (900, 800),
)



draw.rounded_rectangle(
    [1630, 470, 1910, 1050],
    radius=20,
    fill="#F9FAFB",
    outline="#E5E7EB",
    width=2,
)

from views.generate_dashboard.r_donut import ReportDonut 

# generar_donut_and_barra_vertical(
#     img,
#     draw,
#     [
#         Donut(value=21, label="24 Horas"),
#         Donut(value=35, label="48 Horas"),
#         Donut(value=26, label="72 Horas"),
#         Donut(value=43, label="Otros"),
#     ],
#     (1630, 450),
# )
ReportDonut.donut_horizontal(
    img,
    draw,
    [
        Donut(value=21, label="24 Horas"),
        Donut(value=35, label="48 Horas"),
        Donut(value=26, label="72 Horas"),
        Donut(value=43, label="Otros"),
    ],
    (1630, 450),
)


# === Guardar imagen final ===
img.save("dashboard_1920x1080.png", quality=95)
print("Imagen generada: dashboard_1920x1080.png")


ModuleNotFoundError: No module named 'views'

In [90]:
import matplotlib.dates as mdates
import datetime as dt

# ================================
# Datos reales (tú los cambias)
# ================================
fechas = [dt.date(2025, 11, d) for d in range(12, 30)]
pesos = [
    12000,
    11000,
    16000,
    17000,
    10000,
    14000,
    13500,
    13000,
    12500,
    14500,
    9000,
    6000,
    10000,
    8000,
    12000,
    11500,
    7000,
    5000,
]

# Líneas objetivo
meta_alta = 15000  # rojo sutil
meta_media = 10000  # naranja suave

verde_oscuro = "#22C55E"  # Verde principal
verde_claro = "#86EFAC"
amarillo_claro = "#FEF08A"
naranja_suave = "#FDBA74"
rojo_suave = "#FCA5A5"
gris_fondo = "#F9FAFB"
texto_principal = "#1F2937"
texto_secundario = "#6B7280"

# ================================
# Configuración del gráfico
# ================================
plt.rcParams.update(
    {
        "font.family": "Segoe UI",  # o 'Inter', 'SF Pro', 'Arial'
        "font.size": 14,
    }
)

fig, ax = plt.subplots(figsize=(15, 7))
fig.patch.set_facecolor("#FFFFFF")
ax.set_facecolor(gris_fondo)

# Línea principal con degradado visual
line = ax.plot(
    fechas,
    pesos,
    color=verde_oscuro,
    linewidth=4.5,
    marker="o",
    markersize=6,
    markerfacecolor=verde_claro,
    markeredgecolor=verde_oscuro,
    markeredgewidth=2,
)

# Área bajo la curva
ax.fill_between(fechas, pesos, alpha=0.18, color=verde_oscuro)

# Líneas de referencia
ax.axhline(meta_alta, color=rojo_suave, linewidth=2.5, alpha=0.8, linestyle="--")
ax.text(
    fechas[0],
    meta_alta + 600,
    "Meta Alta: 15,000 kg",
    color="#DC2626",
    fontsize=11,
    fontweight="bold",
)

ax.axhline(meta_media, color=naranja_suave, linewidth=2.5, alpha=0.8, linestyle="--")
ax.text(
    fechas[0],
    meta_media + 600,
    "Meta Media: 10,000 kg",
    color="#F97316",
    fontsize=11,
    fontweight="bold",
)

# Título grande y limpio
ax.set_title(
    "CARGA LABORAL POR TONELADAS",
    fontsize=18,
    fontweight="bold",
    color=texto_principal,
    pad=30,
)

# Etiquetas
ax.set_ylabel(
    "Peso Total Despachado (kg)", fontsize=16, color=texto_secundario, labelpad=15
)
ax.set_xlabel("Noviembre 2025", fontsize=16, color=texto_secundario, labelpad=15)

# Formato del eje X (día + mes abreviado, como en el dashboard)
ax.xaxis.set_major_formatter(mdates.DateFormatter("%d %b"))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
ax.tick_params(axis="x", which="major", labelsize=14, colors=texto_secundario)
ax.tick_params(axis="y", which="major", labelsize=14, colors=texto_secundario)

# Quitar bordes superiores y derechos (estilo moderno)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_color("#E5E7EB")
ax.spines["bottom"].set_color("#E5E7EB")

# Grid suave
ax.grid(True, color="white", linewidth=1.5, alpha=0.7)
ax.set_axisbelow(True)

# Valor actual destacado (como el "87%" del dashboard)
ultimo_peso = pesos[-1]
ultimo_fecha = fechas[-1]
ax.annotate(
    f"{ultimo_peso:,} kg",
    xy=(ultimo_fecha, ultimo_peso),
    xytext=(10, 15),
    textcoords="offset points",
    fontsize=14,
    fontweight="bold",
    color=verde_oscuro,
    bbox=dict(
        boxstyle="round,pad=0.5",
        facecolor=verde_claro,
        alpha=0.9,
        edgecolor=verde_oscuro,
    ),
)

# Ajustes finales
plt.tight_layout(pad=3.0)

# Guardar con alta calidad
plt.savefig(
    "carga_laboral_close_style.png",
    dpi=72,
    bbox_inches="tight",
    facecolor="#FFFFFF",
    edgecolor="none",
)
plt.close()

print("¡Gráfico generado con estilo Close.com! → carga_laboral_close_style.png")

¡Gráfico generado con estilo Close.com! → carga_laboral_close_style.png
